In [1]:
import casadi
import numpy as np

In [3]:
from casadi import *

"""
Solve the Rosenbrock problem, formulated as the NLP:
minimize     x^2 + 100*z^2
subject to   z+(1-x)^2-y == 0
Joel Andersson, 2015
"""

# Declare variables
x = SX.sym("x")
y = SX.sym("y")
z = SX.sym("z")

# Formulate the NLP
f = x**2 + 100*z**2
g = z + (1-x)**2 - y
nlp = {'x':vertcat([x,y,z]), 'f':f, 'g':g}
  
# Create an NLP solver
solver = nlpsol("solver", "ipopt", nlp)

# Solve the Rosenbrock problem
res = solver({"x0" :[2.5,3.0,0.75],
              "ubg" : 0,
              "lbg" : 0})

# Print solution
print
print "%50s " % "Optimal cost:", res["f"]
print "%50s " % "Primal solution:", res["x"]
print "%50s " % "Dual solution (simple bounds):", res["lam_x"]
print "%50s " % "Dual solution (nonlinear bounds):", res["lam_g"]


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

In [ ]:
from casadi import *
import numpy as NP
import matplotlib.pyplot as plt

nk = 20    # Control discretization
tf = 10.0  # End time

# Declare variables (use scalar graph)
t  = SX.sym("t")    # time
u  = SX.sym("u")    # control
x  = SX.sym("x",3)  # state

# ODE right hand side function
rhs = vertcat([(1 - x[1]*x[1])*x[0] - x[1] + u, \
               x[0], \
               x[0]*x[0] + x[1]*x[1] + u*u])
f = SXFunction('f', [t,x,u],[rhs])

# Objective function (meyer term)
m = SXFunction('m', [t,x,u],[x[2]])

# Control bounds
u_min = -0.75
u_max = 1.0
u_init = 0.0

u_lb = NP.array([u_min])
u_ub = NP.array([u_max])
u_init = NP.array([u_init])

# State bounds and initial guess
x_min =  [-inf, -inf, -inf]
x_max =  [ inf,  inf,  inf]
xi_min = [ 0.0,  1.0,  0.0]
xi_max = [ 0.0,  1.0,  0.0]
xf_min = [ 0.0,  0.0, -inf]
xf_max = [ 0.0,  0.0,  inf]
x_init = [ 0.0,  0.0,  0.0]

# Dimensions
nx = 3
nu = 1

# Choose collocation points
tau_root = collocationPoints(3,"radau")

# Degree of interpolating polynomial
d = len(tau_root)-1

# Size of the finite elements
h = tf/nk

# Coefficients of the collocation equation
C = NP.zeros((d+1,d+1))

# Coefficients of the continuity equation
D = NP.zeros(d+1)

# Dimensionless time inside one control interval
tau = SX.sym("tau")
  
# All collocation time points
T = NP.zeros((nk,d+1))
for k in range(nk):
  for j in range(d+1):
    T[k,j] = h*(k + tau_root[j])

# For all collocation points
for j in range(d+1):
  # Construct Lagrange polynomials to get the polynomial basis at the collocation point
  L = 1
  for r in range(d+1):
    if r != j:
      L *= (tau-tau_root[r])/(tau_root[j]-tau_root[r])
  lfcn = SXFunction('lfcn', [tau],[L])
  
  # Evaluate the polynomial at the final time to get the coefficients of the continuity equation
  [D[j]] = lfcn([1.0])

  # Evaluate the time derivative of the polynomial at all collocation points to get the coefficients of the continuity equation
  tfcn = lfcn.tangent()
  for r in range(d+1):
    C[j,r], _ = tfcn([tau_root[r]])

# Total number of variables
NX = nk*(d+1)*nx      # Collocated states
NU = nk*nu              # Parametrized controls
NXF = nx                # Final state
NV = NX+NU+NXF

# NLP variable vector
V = MX.sym("V",NV)
  
# All variables with bounds and initial guess
vars_lb = NP.zeros(NV)
vars_ub = NP.zeros(NV)
vars_init = NP.zeros(NV)
offset = 0

# Get collocated states and parametrized control
X = NP.resize(NP.array([],dtype=MX),(nk+1,d+1))
U = NP.resize(NP.array([],dtype=MX),nk)
for k in range(nk):  
  # Collocated states
  for j in range(d+1):
    # Get the expression for the state vector
    X[k,j] = V[offset:offset+nx]
    
    # Add the initial condition
    vars_init[offset:offset+nx] = x_init
    
    # Add bounds
    if k==0 and j==0:
      vars_lb[offset:offset+nx] = xi_min
      vars_ub[offset:offset+nx] = xi_max
    else:
      vars_lb[offset:offset+nx] = x_min
      vars_ub[offset:offset+nx] = x_max
    offset += nx
  
  # Parametrized controls
  U[k] = V[offset:offset+nu]
  vars_lb[offset:offset+nu] = u_min
  vars_ub[offset:offset+nu] = u_max
  vars_init[offset:offset+nu] = u_init
  offset += nu
  
# State at end time
X[nk,0] = V[offset:offset+nx]
vars_lb[offset:offset+nx] = xf_min
vars_ub[offset:offset+nx] = xf_max
vars_init[offset:offset+nx] = x_init
offset += nx
  
# Constraint function for the NLP
g = []
lbg = []
ubg = []

# For all finite elements
for k in range(nk):
  
  # For all collocation points
  for j in range(1,d+1):
        
    # Get an expression for the state derivative at the collocation point
    xp_jk = 0
    for r in range (d+1):
      xp_jk += C[r,j]*X[k,r]
      
    # Add collocation equations to the NLP
    [fk] = f.call([T[k,j], X[k,j], U[k]])
    g.append(h*fk - xp_jk)
    lbg.append(NP.zeros(nx)) # equality constraints
    ubg.append(NP.zeros(nx)) # equality constraints

  # Get an expression for the state at the end of the finite element
  xf_k = 0
  for r in range(d+1):
    xf_k += D[r]*X[k,r]

  # Add continuity equation to NLP
  g.append(X[k+1,0] - xf_k)
  lbg.append(NP.zeros(nx))
  ubg.append(NP.zeros(nx))
  
# Concatenate constraints
g = vertcat(g)

# Objective function
[f] = m.call([T[nk-1,d],X[nk,0],U[nk-1]])
  
# NLP
nlp = MXFunction('nlp', nlpIn(x=V),nlpOut(f=f,g=g))

## ----
## SOLVE THE NLP
## ----

# Set options
opts = {}
opts["expand"] = True
#opts["max_iter"] = 4
opts["linear_solver"] = 'ma27'

# Allocate an NLP solver
solver = NlpSolver("solver", "ipopt", nlp, opts)
arg = {}
  
# Initial condition
arg["x0"] = vars_init

# Bounds on x
arg["lbx"] = vars_lb
arg["ubx"] = vars_ub

# Bounds on g
arg["lbg"] = NP.concatenate(lbg)
arg["ubg"] = NP.concatenate(ubg)

# Solve the problem
res = solver(arg)

# Print the optimal cost
print "optimal cost: ", float(res["f"])

# Retrieve the solution
v_opt = NP.array(res["x"])

# Get values at the beginning of each finite element
x0_opt = v_opt[0::(d+1)*nx+nu]
x1_opt = v_opt[1::(d+1)*nx+nu]
x2_opt = v_opt[2::(d+1)*nx+nu]
u_opt = v_opt[(d+1)*nx::(d+1)*nx+nu]
tgrid = NP.linspace(0,tf,nk+1)
tgrid_u = NP.linspace(0,tf,nk)

# Plot the results
plt.figure(1)
plt.clf()
plt.plot(tgrid,x0_opt,'--')
plt.plot(tgrid,x1_opt,'-.')
plt.step(tgrid_u,u_opt,'-')
plt.title("Van der Pol optimization")
plt.xlabel('time')
plt.legend(['x[0] trajectory','x[1] trajectory','u trajectory'])
plt.grid()
plt.show()

In [ ]:
from casadi import *

# In this example, we fit a nonlinear model to measurements
#
# This example uses more advanced constructs than the vdp* examples:
# Since the number of control intervals is potentially very large here,
# we use memory-efficient Map and MapAccum, in combination with
# codegeneration.
#
# We will be working with a 2-norm objective:
# || y_measured - y_simulated ||_2^2
#
# This form is well-suited for the Gauss-Newton Hessian approximation.

import time

############ SETTINGS #####################
N = 10000  # Number of samples
fs = 610.1 # Sampling frequency [hz]

param_truth = DMatrix([5.625e-6,2.3e-4,1,4.69])
param_guess = DMatrix([5,2,1,5])
scale = vertcat([1e-6,1e-4,1,1])

############ MODELING #####################
y  = MX.sym('y')
dy = MX.sym('dy')
u  = MX.sym('u')

states = vertcat([y,dy]);
controls = u;

M = MX.sym("x")
c = MX.sym("c")
k = MX.sym("k")
k_NL = MX.sym("k_NL")

params = vertcat([M,c,k,k_NL])

rhs = vertcat([dy , (u-k_NL*y**3-k*y-c*dy)/M])

# Form an ode function
ode = MXFunction('ode',[states,controls,params],[rhs])

############ Creating a simulator ##########
N_steps_per_sample = 12
dt = 1/fs/N_steps_per_sample

# Build an integrator for this system: Runge Kutta 4 integrator
[k1] = ode([states,controls,params])
[k2] = ode([states+dt/2.0*k1,controls,params])
[k3] = ode([states+dt/2.0*k2,controls,params])
[k4] = ode([states+dt*k3,controls,params])

states_final = states+dt/6.0*(k1+2*k2+2*k3+k4)

# Create a function that simulates one step propagation in a sample
one_step = MXFunction('one_step',[states, controls, params],[states_final]);

X = states;

for i in range(N_steps_per_sample):
  [X] = one_step([X, controls, params])

# Create a function that simulates all step propagation on a sample
one_sample = MXFunction('one_sample',[states, controls, params], [X])

# speedup trick: expand into scalar operations
one_sample = one_sample.expand()

############ Simulating the system ##########

all_samples = one_sample.mapaccum("all_samples", N)

# Choose an excitation signal
numpy.random.seed(0)
u_data = DMatrix(0.1*numpy.random.random(N))

x0 = DMatrix([0,0])
[X_measured] = all_samples([x0, u_data, repmat(param_truth,1,N) ])

y_data = X_measured[0,:].T

# You may add some noise here
#y_data+= 0.001*numpy.random.random(N)
# When noise is absent, the fit will be perfect.

# Use just-in-time compilation to speed up the evaluation
if Compiler.hasPlugin('clang'):
  opts = {'jit':True, "jit_options":{"flags":['-O3']}}
else:
  print "WARNING; running without jit. This may result in very slow evaluation times"
  opts = {}

############ Create a Gauss-Newton solver ##########
def gauss_newton(e,nlp,V):
  gradF = nlp.gradient()
  jacG = nlp.jacobian("x","g")

  gradF.derivative(0, 1)

  J = jacobian(e,V)
  sigma = MX.sym("sigma")
  hessLag = MXFunction('H',hessLagIn(x=V,lam_f=sigma),hessLagOut(hess=sigma*mul(J.T,J)),opts)
  
  return NlpSolver("solver","ipopt", nlp, {"hess_lag":hessLag})

############ Identifying the simulated system: single shooting strategy ##########

t0 = time.time()

# Note, it is in general a good idea to scale your decision variables such
# that they are in the order of ~0.1..100
[X_symbolic] = all_samples([x0, u_data, repmat(params*scale,1,N) ])

e = y_data-X_symbolic[0,:].T;
nlp = MXFunction("nlp", nlpIn(x=params), nlpOut(f=0.5*inner_prod(e,e)),opts)

solver = gauss_newton(e,nlp, params)
t0 = time.time()-t0

sol = solver(x0=param_guess)

print sol["x"]*scale

assert(max(fabs(sol["x"]*scale-param_truth))<1e-8)

print "Init time [s]: ", t0
############ Identifying the simulated system: multiple shooting strategy ##########

t0 = time.time()
# All states become decision variables
X = MX.sym("X", 2, N)

[Xn] = one_sample.map([X, u_data.T, repmat(params*scale,1,N)], 'openmp')

gaps = Xn[:,:-1]-X[:,1:]

e = y_data-Xn[0,:].T

V = veccat([params, X])

nlp = MXFunction("nlp", nlpIn(x=V), nlpOut(f=0.5*inner_prod(e,e),g=gaps),opts)

# Multipleshooting allows for careful initialization
yd = np.diff(y_data,axis=0)*fs
X_guess = horzcat([ y_data , vertcat([yd,yd[-1]]) ]).T;

x0 = veccat([param_guess,X_guess])

solver = gauss_newton(e,nlp, V)
t0 = time.time()-t0

sol = solver(x0=x0,lbg=0,ubg=0)

print sol["x"][:4]*scale

assert(max(fabs(sol["x"][:4]*scale-param_truth))<1e-8)
print "Init time [s]: ", t0

############ Identifying the simulated system: direct collocation strategy ##########

t0 = time.time()

tau_root = collocationPoints(5,"radau")

# Degree of interpolating polynomial
d = len(tau_root)-1

X = MX.sym("X",2*(d+1),N)
Xlast = MX.sym("Xlast",2)

G = collocationResidual(ode,tau_root)
G = G.expand()

all_G = Map("all_G",G,N,[True,True,True,False,False],[True,True])

X0,Xhelper = vertsplit2(X,2)
eq,Xp = all_G([Xhelper,X0,u_data.T,params*scale,1.0/fs])

Xn = horzsplit2(horzcat([X0,Xlast]),1)[1]

g = vertcat([eq,Xp-Xn])

e = y_data-vertsplit(Xn)[0].T

V = veccat([params,X,Xlast])

nlp = MXFunction("nlp", nlpIn(x=V), nlpOut(f=0.5*inner_prod(e,e),g=g),opts)

# Multipleshooting allows for careful initialization
yd = np.diff(y_data,axis=0)*fs
X_guess = horzcat([ y_data , vertcat([yd,yd[-1]]) ]).T

x0 = veccat([param_guess, repmat(X_guess,d+1,1), X_guess[:,-1] ])

solver = gauss_newton(e,nlp, V)
t0 = time.time()-t0

sol = solver(x0=x0,lbg=0,ubg=0)

print sol["x"][:4]*scale

assert(max(fabs(sol["x"][:4]*scale-param_truth))<1e-5)
print "Init time [s]: ", t0


In [ ]:
#
#     This file is part of CasADi.
#
#     CasADi -- A symbolic framework for dynamic optimization.
#     Copyright (C) 2010-2014 Joel Andersson, Joris Gillis, Moritz Diehl,
#                             K.U. Leuven. All rights reserved.
#     Copyright (C) 2011-2014 Greg Horn
#
#     CasADi is free software; you can redistribute it and/or
#     modify it under the terms of the GNU Lesser General Public
#     License as published by the Free Software Foundation; either
#     version 3 of the License, or (at your option) any later version.
#
#     CasADi is distributed in the hope that it will be useful,
#     but WITHOUT ANY WARRANTY; without even the implied warranty of
#     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
#     Lesser General Public License for more details.
#
#     You should have received a copy of the GNU Lesser General Public
#     License along with CasADi; if not, write to the Free Software
#     Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301  USA
#
#

# For documentation about this examples, check http://docs.casadi.org/documents/mhe_spring_damper.pdf

from casadi import *
import numpy as NP
import matplotlib.pyplot as plt
import time
from casadi.tools import *
from scipy import linalg, matrix
plt.interactive(True)

NP.random.seed(0)

# Settings of the filter
N = 10 # Horizon length
dt = 0.05; # Time step

sigma_p = 0.005 # Standard deviation of the position measurements
sigma_w = 0.1 # Standard deviation for the process noise
R = DM(1/sigma_p**2) # resulting weighting matrix for the position measurements
Q = DM(1/sigma_w**2) # resulting weighting matrix for the process noise

Nsimulation = 1000 # Lenght of the simulation

# Parameters of the system
m = 1 # The weight of the mass
k = 1 # The spring constant
c = 0.5 # The damping of the system
# The state
states = struct_symSX(["x","dx"]) # Full state vector of the system: position x and velocity dx
Nstates = states.size # Number of states
# Set up some aliases
x,dx = states[...]

# The control input
controls = struct_symSX(["F"]) # Full control vector of the system: Input force F
Ncontrols = controls.size # Number of control inputs
# Set up some aliases
F, = controls[...]

# Disturbances
disturbances = struct_symSX(["w"]) # Process noise vector
Ndisturbances = disturbances.size # Number of disturbances
# Set up some aliases
w, = disturbances[...]

# Measurements
measurements = struct_symSX(["y"]) # Measurement vector
Nmeas = measurements.size # Number of measurements
# Set up some aliases
y, = measurements[...]

# Create Structure for the entire horizon

# Structure that will be degrees of freedom for the optimizer
shooting = struct_symSX([(entry("X",repeat=N,struct=states),entry("W",repeat=N-1,struct=disturbances))])
# Structure that will be fixed parameters for the optimizer
parameters = struct_symSX([(entry("U",repeat=N-1,struct=controls),entry("Y",repeat=N,struct=measurements),entry("S",shape=(Nstates,Nstates)),entry("x0",shape=(Nstates,1)))])
S = parameters["S"]
x0 = parameters["x0"]
# Define the ODE right hand side
rhs = struct_SX(states)
rhs["x"] = dx
rhs["dx"] = (-k*x-c*dx+F)/m+w

f = Function('f', [states,controls,disturbances],[rhs])

# Build an integrator for this system: Runge Kutta 4 integrator
k1 = f([states,controls,disturbances])[0]
k2 = f([states+dt/2.0*k1,controls,disturbances])[0]
k3 = f([states+dt/2.0*k2,controls,disturbances])[0]
k4 = f([states+dt*k3,controls,disturbances])[0]

states_1 = states+dt/6.0*(k1+2*k2+2*k3+k4)
phi = Function('phi', [states,controls,disturbances],[states_1])
PHI = phi.jacobian()
# Define the measurement system
h = Function('h', [states],[x]) # We have measurements of the position
H = h.jacobian()
# Build the objective
obj = 0
# First the arrival cost
obj += mtimes([(shooting["X",0]-parameters["x0"]).T,S,(shooting["X",0]-parameters["x0"])])
#Next the cost for the measurement noise
for i in range(N):
  vm = h([shooting["X",i]])[0]-parameters["Y",i]
  obj += mtimes([vm.T,R,vm])
#And also the cost for the process noise
for i in range(N-1):
  obj += mtimes([shooting["W",i].T,Q,shooting["W",i]])

# Build the multiple shooting constraints
g = []
for i in range(N-1):
  g.append( shooting["X",i+1] - phi([shooting["X",i],parameters["U",i],shooting["W",i]])[0] )

# Formulate the NLP
nlp = {'x':shooting, 'p':parameters, 'f':obj, 'g':vertcat(g)}

# Make a simulation to create the data for the problem
simulated_X = DM.zeros(Nstates,Nsimulation)
simulated_X[:,0] = DM([1,0]) # Initial state
t = NP.linspace(0,(Nsimulation-1)*dt,Nsimulation) # Time grid
simulated_U = DM(cos(t[0:-1])).T # control input for the simulation
simulated_U[:,Nsimulation/2:] = 0.0
simulated_W = DM(sigma_w*NP.random.randn(Ndisturbances,Nsimulation-1)) # Process noise for the simulation
for i in range(Nsimulation-1):
  phires = phi([simulated_X[:,i], simulated_U[:,i], simulated_W[:,i]])
  simulated_X[:,i+1] = phires[0]
#Create the measurements from these states
simulated_Y = DM.zeros(Nmeas,Nsimulation) # Holder for the measurements
for i in range(Nsimulation):
  h_res = h([simulated_X[:,i]])
  simulated_Y[:,i] = h_res[0]
# Add noise the the position measurements
simulated_Y += sigma_p*NP.random.randn(simulated_Y.shape[0],simulated_Y.shape[1])

#The initial estimate and related covariance, which will be used for the arrival cost
sigma_x0 = 0.01
P = sigma_x0**2*DM.eye(Nstates)
x0 = simulated_X[:,0] + sigma_x0*NP.random.randn(Nstates,1)
# Create the solver
opts = {"ipopt.print_level":0, "print_time": False, 'ipopt.max_iter':100}
nlpsol = nlpsol("nlpsol", "ipopt", nlp, opts)

# Create a holder for the estimated states and disturbances
estimated_X= DM.zeros(Nstates,Nsimulation)
estimated_W = DM.zeros(Ndisturbances,Nsimulation-1)
# For the first instance we run the filter, we need to initialize it.
current_parameters = parameters(0)
current_parameters["U",horzcat] = simulated_U[:,0:N-1]
current_parameters["Y",horzcat] = simulated_Y[:,0:N]
current_parameters["S"] = linalg.inv(P) # Arrival cost is the inverse of the initial covariance
current_parameters["x0"] = x0
initialisation_state = shooting(0)
initialisation_state["X",horzcat] = simulated_X[:,0:N]
res = nlpsol({"p":current_parameters, "x0":initialisation_state, "lbg":0, "ubg":0})

# Get the solution
solution = shooting(res["x"])
estimated_X[:,0:N] = solution["X",horzcat]
estimated_W[:,0:N-1] = solution["W",horzcat]

# Now make a loop for the rest of the simulation
for i in range(1,Nsimulation-N+1):

  # Update the arrival cost, using linearisations around the estimate of MHE at the beginning of the horizon (according to the 'Smoothed EKF Update'): first update the state and covariance with the measurement that will be deleted, and next propagate the state and covariance because of the shifting of the horizon
  print "step %d/%d (%s)" % (i, Nsimulation-N , nlpsol.stats()["return_status"])
  H0 = H([solution["X",0]])[0]
  K = mtimes([P,H0.T,linalg.inv(mtimes([H0,P,H0.T])+R)])
  P = mtimes((DM.eye(Nstates)-mtimes(K,H0)),P)
  [h0] = h([solution["X",0]])
  x0 = x0 + mtimes(K, current_parameters["Y",0]-h0-mtimes(H0,x0-solution["X",0]))
  [x0] = phi([x0, current_parameters["U",0], solution["W",0]])
  F = PHI([solution["X",0], current_parameters["U",0], solution["W",0]])[0]
  P = mtimes([F,P,F.T]) + linalg.inv(Q)
  # Get the measurements and control inputs 
  current_parameters["U",horzcat] = simulated_U[:,i:i+N-1]
  current_parameters["Y",horzcat] = simulated_Y[:,i:i+N]
  current_parameters["S"] = linalg.inv(P)
  current_parameters["x0"] = x0
  # Initialize the system with the shifted solution
  initialisation_state["W",horzcat,0:N-2] = estimated_W[:,i:i+N-2] # The shifted solution for the disturbances
  initialisation_state["W",N-2] = DM.zeros(Ndisturbances,1) # The last node for the disturbances is initialized with zeros
  initialisation_state["X",horzcat,0:N-1] = estimated_X[:,i:i+N-1] # The shifted solution for the state estimates
  # The last node for the state is initialized with a forward simulation
  [phi0] = phi([initialisation_state["X",N-1], current_parameters["U",-1], initialisation_state["W",-1]])
  initialisation_state["X",N-1] = phi0
  # And now initialize the solver and solve the problem
  res = nlpsol({"p":current_parameters, "x0":initialisation_state, "lbg":0, "ubg":0})
  solution = shooting(res["x"])

  # Now get the state estimate. Note that we are only interested in the last node of the horizon
  estimated_X[:,N-1+i] = solution["X",N-1]
  estimated_W[:,N-2+i] = solution["W",N-2]
# Plot the results
plt.figure(1)
plt.clf()
plt.plot(t,vec(estimated_X[0,:]),'b--')
plt.plot(t,vec(simulated_X[0,:]),'r--')
plt.title("Position")
plt.xlabel('Time')
plt.legend(['Estimated position','Real position'])
plt.grid()

plt.figure(2)
plt.clf()
plt.plot(t,vec(estimated_X[0,:]-simulated_X[0,:]),'b--')
plt.title("Position error")
plt.xlabel('Time')
plt.legend(['Error between estimated and real position'])
plt.grid()

plt.show()

error = estimated_X[0,:]-simulated_X[0,:]
print mtimes(error,error.T)
assert(mtimes(error,error.T)<0.01)

In [ ]:
import casadi.tools as t

In [10]:
from ipywidgets import FloatProgress
from IPython.display import display
from time import sleep
f = FloatProgress(min=0, max=100)
display(f)
f.bar_style = 'success'

In [11]:
for i in xrange(100):
   sleep(0.1)
   f.value = i + 1

In [11]:
import ipywidgets